# Next Word Prediction Model using Python

In [1]:
!pip install nltk

In [2]:
# make next word prediction

In [3]:
#libraries

In [4]:
import nltk
import pandas as pd
import warnings
from nltk.tokenize import sent_tokenize
import numpy as np
import scipy as sp
from nltk import word_tokenize
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk import word_tokenize
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [5]:
#read the data

In [6]:
df = pd.read_csv("sherlock-holm.es_stories_plain-text_advs.txt", sep='\t', header=None, names=['text'])

In [7]:
#EDA

In [8]:
#see first 10 cels

In [9]:
df.head(25)

,text
0,THE ADVENTURES OF SHER...
1,Arthur Conan Doyle
2,Table of contents
3,A Scandal in Bohemia
4,The Red-Headed League
5,A Case of Identity
6,The Boscombe Valley Mystery
7,The Five Orange Pips
8,The Man with the Twisted Lip
9,The Adventure of the Blue Carbu...


In [10]:
#see finaly 10 cels

In [11]:
df.tail(10)

,text
9556,"Walsall, where I believe that she has met..."
9557,----------
9558,"This text is provided to you ""as-is"" with..."
9559,"warranties of any kind, expressed or impl..."
9560,"the text or any medium it may be on, incl..."
9561,warranties of merchantablity or fitness f...
9562,This text was formatted from various free...
9563,See http://sherlock-holm.es for an electr...
9564,additional information about it.
9565,This text comes from the collection's ver...


In [12]:
#see mising values

In [13]:
df.isnull().sum()

text    0
dtype: int64

## Data Preprocesing

In [14]:
#drop first and finaly useless columns

In [15]:
df = df.iloc[21:9550]

In [16]:
# convert to lowercase, remove non-alphanumeric characters, digits, newline, and carriage return

In [17]:
df['text'] = df['text'].str.lower().replace('[^\w\s]', '', regex=True).replace('\d+', '', regex=True).replace('\n', '', regex=True).replace('\r', '', regex=True)

In [18]:
# null chek

In [19]:
df.isnull().sum()

text    0
dtype: int64

In [20]:
df.tail(10)

,text
9540,i am sure we owe you an apology mrs tolle...
9541,have certainly cleared up everything whic...
9542,the country surgeon and mrs rucastle so i...
9543,had best escort miss hunter back to winch...
9544,that our locus standi now is rather a que...
9545,and thus was solved the mystery of the si...
9546,beeches in front of the door mr rucastle ...
9547,broken man kept alive solely through the ...
9548,they still live with their old servants w...
9549,rucastles past life that he finds it diff...


In [21]:
#drop null

In [22]:
df = df[df['text'].str.strip() != '']

In [23]:
df.tail(10)

,text
9540,i am sure we owe you an apology mrs tolle...
9541,have certainly cleared up everything whic...
9542,the country surgeon and mrs rucastle so i...
9543,had best escort miss hunter back to winch...
9544,that our locus standi now is rather a que...
9545,and thus was solved the mystery of the si...
9546,beeches in front of the door mr rucastle ...
9547,broken man kept alive solely through the ...
9548,they still live with their old servants w...
9549,rucastles past life that he finds it diff...


In [24]:
#reset indexes

In [25]:
df.reset_index(drop=True, inplace=True)

In [26]:
df.tail(10)

,text
9519,i am sure we owe you an apology mrs tolle...
9520,have certainly cleared up everything whic...
9521,the country surgeon and mrs rucastle so i...
9522,had best escort miss hunter back to winch...
9523,that our locus standi now is rather a que...
9524,and thus was solved the mystery of the si...
9525,beeches in front of the door mr rucastle ...
9526,broken man kept alive solely through the ...
9527,they still live with their old servants w...
9528,rucastles past life that he finds it diff...


In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
# tokenize the text
df['tokenized_text'] = df['text'].apply(word_tokenize)

In [29]:
df[['tokenized_text']].head()

,tokenized_text
0,"[to, sherlock, holmes, she, is, always, the, w..."
1,"[mention, her, under, any, other, name, in, hi..."
2,"[predominates, the, whole, of, her, sex, it, w..."
3,"[emotion, akin, to, love, for, irene, adler, a..."
4,"[particularly, were, abhorrent, to, his, cold,..."


In [30]:
# flatten the list of tokenized_text
from itertools import chain
all_tokens = list(chain.from_iterable(df['tokenized_text']))

In [31]:
# tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens)
total_words = len(tokenizer.word_index) + 1

In [33]:
from itertools import chain
# apply the tokenization and n-gram sequence
input_sequences = df['tokenized_text'].apply(lambda tokens: [tokens[:i+1] for i in range(1, len(tokenizer.texts_to_sequences([tokens])[0]))])
# flatten the resulting list of lists
input_sequences = list(chain.from_iterable(input_sequences))


In [35]:
# convert the input sequences to integer indices using the tokenizer
input_sequences_int = tokenizer.texts_to_sequences(input_sequences)
# pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences_int])
input_sequences_padded = pad_sequences(input_sequences_int, maxlen=max_sequence_len, padding='pre')
X, y = input_sequences_padded[:, :-1], input_sequences_padded[:, -1]

In [36]:
#model

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# define the model
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# print a summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 50)            415800    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 8316)              839916    
                                                                 
Total params: 1316116 (5.02 MB)
Trainable params: 1316116 (5.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
from tensorflow.keras.utils import to_categorical

# convert labels to categorical format
y_categorical = to_categorical(y, num_classes=total_words)

# train the model
model.fit(X, y_categorical, epochs=100, verbose=1)


Epoch 1/100
2960/2960 [==============================] - 68s 22ms/step - loss: 6.3323 - accuracy: 0.0670
Epoch 2/100
2960/2960 [==============================] - 66s 22ms/step - loss: 5.6631 - accuracy: 0.1087
Epoch 3/100
2960/2960 [==============================] - 66s 22ms/step - loss: 5.3005 - accuracy: 0.1346
Epoch 4/100
2960/2960 [==============================] - 66s 22ms/step - loss: 5.0222 - accuracy: 0.1515
Epoch 5/100
2960/2960 [==============================] - 67s 23ms/step - loss: 4.7900 - accuracy: 0.1642
Epoch 6/100
2960/2960 [==============================] - 70s 24ms/step - loss: 4.5789 - accuracy: 0.1765
Epoch 7/100
2960/2960 [==============================] - 67s 23ms/step - loss: 4.3853 - accuracy: 0.1886
Epoch 8/100
2960/2960 [==============================] - 68s 23ms/step - loss: 4.2032 - accuracy: 0.2021
Epoch 9/100
2960/2960 [==============================] - 67s 23ms/step - loss: 4.0308 - accuracy: 0.2175
Epoch 10/100
2960/2960 [==============================]

In [39]:
# save model

In [40]:
import joblib

In [42]:
joblib.dump(model,"model_n-gram.joblib")

['model_n-gram.joblib']

In [43]:
#sources

-Zafer Acar notes ,-[https://chat.openai.com/]
,-[https://python.plainenglish.io/85-data-science-projects-c03c8750599e]
,-[https://www.geeksforgeeks.org/next-word-prediction-with-deep-learning-in-nlp/]